<a href="https://colab.research.google.com/github/Neha-kumari31/Prediction-of-Cancer-Mortality-Risk-/blob/master/Prediction_of_Cancer_Mortality_Risk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install plotly_express

In [2]:
!pip install category_encoders

     |████████████████████████████████| 102kB 2.3MB/s 


In [3]:
!pip install pdpbox

     |████████████████████████████████| 57.7MB 69kB/s 
  Created wheel for pdpbox: filename=PDPbox-0.2.0-cp36-none-any.whl size=57690723 sha256=bec30e68da57cf9e3633ee50003290b70ab50ba7e5ea47886e47a159c0da1046
  Stored in directory: /root/.cache/pip/wheels/7d/08/51/63fd122b04a2c87d780464eeffb94867c75bd96a64d500a3fe
Successfully built pdpbox


In [4]:
!pip install Shap

     |████████████████████████████████| 266kB 2.4MB/s 
  Created wheel for Shap: filename=shap-0.34.0-cp36-cp36m-linux_x86_64.whl size=383199 sha256=f316826e09c4c8232c5564151bb06499c5642f999dc326cfb1e83c0c9965e48c
  Stored in directory: /root/.cache/pip/wheels/28/7e/28/57bc9fcb77579b0df6561298648249370648c0b91dfe42c8ec
Successfully built Shap


In [5]:
!pip install eli5

     |████████████████████████████████| 112kB 2.7MB/s 


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly_express as px
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.pipeline import make_pipeline
import category_encoders as ce
from scipy.stats import randint, uniform
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
from pdpbox.pdp import pdp_isolate, pdp_plot
import shap
import plotly.graph_objects as go 
from sklearn.metrics import classification_report, confusion_matrix
import eli5
from eli5.sklearn import PermutationImportance


In [0]:
from sklearn.datasets import load_iris
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, cross_val_score, KFold
import numpy as np
import category_encoders as ce
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from xgboost import XGBClassifier


In [8]:
df = pd.read_csv("final_data.csv", index_col = 0)
df.fillna(method='ffill', inplace = True)
#Split the data into train and test
from sklearn.model_selection import train_test_split
train , test = train_test_split(df, train_size = 0.80, test_size= 0.20, stratify= df['MortalityRisk'], random_state= 42)

#train, val = train_test_split(train_val, test_size = 0.20, stratify= train_val['MortalityRisk'] ,random_state= 42)

# wrangle data

def wrangle(X):
    """Wrangle train, validate, and test sets in the same way"""
    
    # Prevent SettingWithCopyWarning
    X = X.copy()
    X = X[X['Age_Adjusted_MortalityRate'] <= np.percentile(X['Age_Adjusted_MortalityRate'], 99.5)]
    
    # 'Recent5yrdeathrate' column has highest feature importance
    # this look like a leakage, let's drop it
    X = X.drop(columns=['Age_Adjusted_MortalityRate','Recent5yrdeathrate','AvgAnnualCount','County','CountyCode'])
    
    # Convert Zipcode and CountyCode as object

    #X['CountyCode'] = X['CountyCode'].astype('str')
    
    #Drop the high cardinality features
    X = X.drop(columns =[ 'CountyState'])
    
     # Reset index
    X = X.reset_index(drop=True)
    
    return X
train = wrangle(train)
test = wrangle(test)
#val = wrangle(val)
train.shape, test.shape, train.columns

FileNotFoundError: ignored

In [0]:
target = 'MortalityRisk'
features = train.drop(columns= target).columns

X_train= train[features]
y_train = train[target]

X_test = test[features]
y_test= test[target]
X_train.shape, X_test.shape

##Baseline Logistic Model

In [0]:
#with numerical features only
from sklearn.linear_model import LogisticRegression
def LReg(X_train, y_train):
    
    X_train = X_train.select_dtypes(include='number')
    model = LogisticRegression()
    model.fit(X_train, y_train)
    k = 5
    LRscores = cross_val_score(model, X_train, y_train, cv=k, scoring = 'roc_auc' )
    print(f'Validation Acuuracy for {k} folds:', LRscores.mean())
    
    
    
    
LReg(X_train, y_train)

In [0]:
import category_encoders as ce
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import Ridge
lr_pipeline = make_pipeline(
    ce.OrdinalEncoder(),
    StandardScaler(),
    LogisticRegression(multi_class='auto', solver='lbfgs', n_jobs=-1, max_iter= 1000)
)

# # Fit on train
lr_pipeline.fit(X_train, y_train)
# print('Training Accuracy', pipeline.score(X_train, y_train))
k = 10
LRscores = cross_val_score(lr_pipeline, X_train, y_train, cv=k, scoring = 'accuracy' )
print(f'Validation Acuuracy for {k} folds:', LRscores.mean())

In [0]:
gb_pipeline = make_pipeline(
    ce.OrdinalEncoder(),
    StandardScaler(),
    XGBClassifier(n_estimators=50,    
                      max_depth=9,          
                      learning_rate=0.1,
                      min_child_weight = 13,
                      max_features = 25,
                      random_state=42,
                      n_jobs=-1)
)
gb_pipeline.fit(X_train, y_train)
k = 5
scores = cross_val_score(gb_pipeline, X_train, y_train, cv=k, 
                         scoring='accuracy')
print(f'ROC AUC Score for {k} folds:', scores.mean())


In [0]:
y_predict = gb_pipeline.predict(X_test)
Xgb_cv_score = cross_val_score(gb_pipeline, X_train, y_train, cv=k, scoring ='roc_auc')

print("=== Confusion Matrix ===")
print(confusion_matrix(y_test, y_predict))
print('\n')
print("=== Classification Report ===")
print(classification_report(y_test, y_predict))
print('\n')
print("=== All AUC Scores ===")
print(Xgb_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - XGB Classifier: ", Xgb_cv_score.mean())

In [0]:
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels

def plot_confusion_matrix(y_true, y_pred):
    
    # Get label names from y target vector
    labels = unique_labels(y_true)
    
    # Set column labels for confusion matrix
    columns = [f'Predicted {label}' for label in labels]
    # Set index labels for confusion matrix
    index = [f'Actual {label}' for label in labels]
    
    # Create pandas dataframe to store info prior to plotting
    table = pd.DataFrame(confusion_matrix(y_true, y_pred),
                         columns=columns,
                         index=index)
    
    return sns.heatmap(table, annot=True, fmt='d', cmap='YlGnBu')

In [0]:
plt.figure(figsize=(6,6))
plot_confusion_matrix(y_test, y_predict);

In [0]:
y_test= y_test.replace({'High': 1, 'low':0})

In [0]:
# Use scikit-learn to calculate TPR & FPR at various thresholds
from sklearn.metrics import roc_curve
y_pred_proba = gb_pipeline.predict_proba(X_test)[:, 0] # Probability for last class
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)

In [0]:
# See the results in a table
pd.DataFrame({
    'False Positive Rate': fpr, 
    'True Positive Rate': tpr, 
    'Threshold': thresholds
})

In [0]:
import matplotlib.pyplot as plt
plt.plot(fpr, tpr)
plt.title('ROC curve')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate');

In [0]:
# Let's do a permutation importance to see just what the most important features are
%matplotlib inline
import matplotlib.pyplot as plt
# Get feature importances
xgb_model =gb_pipeline.named_steps['xgbclassifier']
xgb_encoder = gb_pipeline.named_steps['ordinalencoder']
importances = pd.Series(xgb_model.feature_importances_, X_train.columns)
# Plot feature importances
n=10
plt.figure(figsize=(7, 7))
plt.title('Feature Importance Graph')
importances.sort_values()[-n:].plot.barh(color='grey');

In [0]:
import eli5
from eli5.sklearn import PermutationImportance

X_train_encoded =xgb_encoder.transform(X_train)
X_test_encoded = xgb_encoder.transform(X_test)

permuter = PermutationImportance(
    gb_pipeline,
    scoring='roc_auc',
    n_iter=10,
    random_state=42
)
permuter.fit(X_test_encoded, y_test)
eli5.show_weights(
    permuter,
    top=None,
    feature_names=X_test.columns.tolist()
)

In [0]:
feature = 'PopwithHighSchoolDegree'

isolated = pdp_isolate(
    model = gb_pipeline,
    dataset=X_test,
    model_features=X_test.columns,
    feature=feature,
)

pdp_plot(isolated, feature_name=feature)

In [0]:
plt.rcParams['figure.dpi'] = 72

feature = 'MedHHInc'

isolated = pdp_isolate(
    model = gb_pipeline,
    dataset=X_test,
    model_features=X_test.columns,
    feature=feature,
)

pdp_plot(isolated, feature_name=feature)

In [0]:
grid = isolated.feature_grids
pdp = isolated.pdp
plt.figure(figsize=(10,6))
plt.plot(grid, pdp)
plt.ylabel('Mortality Risk')
plt.xlabel('%Median HouseHold Income')
plt.title('Partial Dependence Plot for Median HouseHold Income');

In [0]:
plt.rcParams['figure.dpi'] = 72

feature = 'PovertyPct'

isolated= pdp_isolate(
    model = gb_pipeline,
    dataset=X_test,
    model_features=X_test.columns,
    feature=feature
)

pdp_plot(isolated, feature_name=feature)


In [0]:
grid = isolated.feature_grids
pdp = isolated.pdp
plt.figure(figsize=(10,6))
plt.plot(grid, pdp)
plt.ylabel('Mortality Risk')
plt.xlabel('%Median HouseHold Income')
plt.title('Partial Dependence Plot for Median HouseHold Income');

In [0]:
X_test.iloc[50]

In [0]:
y_test.iloc[1]

In [0]:
y_pred = gb_pipeline.predict(X_test)

In [0]:
y_pred

In [0]:
xgb_model =gb_pipeline.named_steps['xgbclassifier']
xgb_encoder = gb_pipeline.named_steps['ordinalencoder']
row = X_test.iloc[[1]]

explainer = shap.TreeExplainer(xgb_model)
row_encoded = xgb_encoder.transform(row)
shap_values = explainer.shap_values(row_encoded)

shap.initjs()
shap.force_plot(
    base_value=explainer.expected_value,
    shap_values=shap_values,
    features=row
)

In [0]:
xgb_model =gb_pipeline.named_steps['xgbclassifier']
xgb_encoder = gb_pipeline.named_steps['ordinalencoder']
row = X_test.iloc[[5]]

explainer = shap.TreeExplainer(xgb_model)
row_encoded = xgb_encoder.transform(row)
shap_values = explainer.shap_values(row_encoded)

shap.initjs()
shap.force_plot(
    base_value=explainer.expected_value,
    shap_values=shap_values,
    features=row
)

In [0]:


explainer = shap.TreeExplainer(xgb_model)
# row_encoded = encoder.transform(row)
# shap_values = explainer.shap_values(row_encoded)

# shap.initjs()
# shap.force_plot(
#     base_value=explainer.expected_value,
#     shap_values=shap_values,
#     features=row
# )

In [0]:
import shap
row = X_test.iloc[[50]]
processor = gb_pipeline[:-1]
row_transformed = processor.transform(row)

explainer = shap.TreeExplainer(gb_pipeline[1])
row_processed = row_transformed
shap_values = explainer.shap_values(row_processed)

shap.initjs()
shap.force_plot(
    base_value=explainer.expected_value[1], 
    shap_values=shap_values[1], 
    features=row
)
